In [1]:
import os
import pandas as pd
import numpy as np
import random
import operator
import functools
import pickle
import re

In [2]:
pd.set_option('display.max_columns', None)
from IPython.display import display

### poverty rates

In [5]:
work_dir = 'I:/NCES/NCES_Dev/sunjoo_LEE_MOVE/CRDC_outlier_2021_2/0_processed_data/'

In [6]:
# read-in data R-prepped data
dfa = pd.read_csv('{}crdc_prepped.csv'.format(work_dir), dtype={'leaid':str, 'combokey':str})

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (794,796,797) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
# import LEA poverty data for 2020
pov = pd.read_excel('{}ussd20.xls'.format(work_dir), skiprows=[0,1], dtype={'State FIPS Code':str, 'District ID':str})
pov.columns = ['state','state_fips','district', 'name','population','pop5-17','pov5-17']

In [8]:
# get poverty rate by lea and merge into original data
pov['leaid'] = pov['state_fips'] + pov['district']
pov['pov_per_5-17'] = pov['pov5-17'] / pov['pop5-17']
dfa = pd.merge(dfa,pov[['leaid','pov_per_5-17']], how='left', on='leaid')

In [9]:
# for LEAs without poverty rate, fill in with average of the state
pv = pov[['state','pov5-17']].groupby('state').sum()
pop = pov[['state','pop5-17']].groupby('state').sum()
st_avg = pv.reset_index()['pov5-17'] / pop.reset_index()['pop5-17']
st_avg = pd.concat([pv.reset_index(), st_avg], axis=1)
dfa = pd.merge(dfa, st_avg[['state',0]], how='left')
dfa['pov_per_5-17'] = dfa.apply(lambda row: row['pov_per_5-17'] if not np.isnan(row['pov_per_5-17']) else row[0], axis=1)
dfa.drop([0], axis=1, inplace=True)

In [10]:
# for schools with no state, use national average
dfa.loc[np.isnan(dfa['pov_per_5-17']), 'pov_per_5-17'] = pov['pov5-17'].sum() / pov['pop5-17'].sum()

In [11]:
dfa = dfa.drop(['leaid', 'sch_name', 'state', 'combokey'], axis=1)

In [12]:
dfa.to_csv('{}crdc_prepped_formod.csv'.format(work_dir), index=False)